In [7]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver
import os

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘

corp_code = '348350'
start = '2020-01-02'
dart_list = dart.list(corp_code,start=start)
lists = dart_list[dart_list['report_nm'].str.contains('공급계약체결')&~dart_list['report_nm'].str.contains('기재정정')]
print(f'{len(lists)}개 조회')
def supply(rcp_no,corp):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt[[1,2]]
    tt = tt.T
    
    if '내용' in tt.iloc[0,0]: # 공급계약은 두가지 타입으로 존재
        tt = tt[[19,0,2,5,6,7,12,13,14,21,20]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    else:
        tt = tt[[12,1,2,3,4,6,8,9,10,13,14]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    tt.insert(0,'기업',corp)
    time.sleep(1)
    return tt


df = pd.DataFrame()

for i in range(len(lists)):
    try:
        rcp_no = lists['rcept_no'].values[i]
        corp = lists['flr_nm'].values[i]
        supply_df = supply(rcp_no,corp)
        
        print(f'{i}_{corp} 완료')  
        df = pd.concat([df,supply_df])
    except Exception as e:
        print(f'{corp}',e)


folder = f'공급계약_개별기업'
file_name = f'공급계약_{start}부터_{corp_code}1.xlsx'
file_path = os.path.join(folder,file_name)


if not os.path.exists(folder):
    os.mkdir(folder)
    
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        df.to_excel(file_path)



9개 조회
0_위드텍 완료
1_위드텍 완료
2_위드텍 완료
3_위드텍 완료
4_위드텍 완료
5_위드텍 완료
6_위드텍 완료
7_위드텍 완료
8_위드텍 완료


In [8]:
df

,기업,계약일자,계약내용,계약금액(원),최근매출액(원),매출액대비(%),계약상대,판매공급지역,시작일,종료일,유보사유,유보기한
0,위드텍,2024-09-30,TMS(산업배출가스측정)유지보수업무 위탁계약의 건,"11,458,037,670","49,561,291,840",23.12,삼성전자(주),대한민국,2024-10-01,2025-09-30,-,-
0,위드텍,2023-12-27,AMC 설비 유지보수 연간 계약의 건,"3,736,850,400","72,605,947,195",5.15,삼성전자(주),대한민국,2024-01-01,2024-12-31,- 당사의 최근 매출액은 2022년도 연결기준 매출액입니다. - 계약금액은 부가가...,9. 기타 투자판단에 참고할 사항
0,위드텍,2023-09-27,TMS(산업배출가스측정)유지보수업무 위탁계약의 건,"9,758,946,670","72,605,947,195",13.44,삼성전자(주),대한민국,2023-10-01,2024-09-30,-,-
0,위드텍,2022-03-17,반도체 제조환경 및 공정모니터닝 장비 공급계약의 건(NAVI-TM100L 등),"5,470,315,000","55,617,176,046",9.84,SK hynix,대한민국,2022-03-17,2022-09-30,- 당사의 최근매출액은 2020년도 연결기준 매출액입니다. - 계약금액은 부가가치...,9. 기타 투자판단에 참고할 사항
0,위드텍,2022-01-27,반도체 제조환경(AMC)장비 공급계약의 건,"7,553,500,000","55,617,176,046",13.58,엘지디스플레이(주),대한민국,2022-01-27,2022-09-30,-,-
0,위드텍,2021-09-17,반도체 제조환경(AMC)모니터링 장비 공급계약의 건,"2,990,000,000","55,617,176,046",5.38,엘지디스플레이(주),대한민국,2021-09-17,2022-03-15,- 당사의 최근매출액은 2020년도 연결기준 매출액입니다. - 계약금액은 부가가치...,9. 기타 투자판단에 참고할 사항
0,위드텍,2021-06-30,TMS(산업배출가스측정) 유지보수업무 위탁계약의 건,"6,054,197,919","55,617,176,046",10.89,삼성전자(주),대한민국,2021-07-01,2022-06-30,-,-
0,위드텍,2021-02-02,반도체 제조환경(AMCs 등)장비 공급계약의 건,"2,330,000,000","58,862,418,665",3.96,SK Hynix,대한민국,2021-02-02,2021-06-30,- 당사의 최근매출액은 2019년도 연결기준 매출액입니다. - 계약금액은 부가...,9. 기타 투자판단에 참고할 사항
0,위드텍,2020-11-25,반도체 제조환경(AMCs 등)장비 공급계약의 건,"10,613,997,000","58,862,418,665",18.03,SK hynix,대한민국,2020-11-25,2021-03-31,-,-


""
